In [ ]:
import { display } from "tslab";
import { readFileSync } from "fs";

const css = readFileSync("../style.css", "utf8");
display.html(`<style>${css}</style>`);

# Minimizing a Finite State Machine

In this notebook, we show how a **DFA** (Deterministic Finite Automaton) can be minimized by identifying states that are equivalent.

We will strictly adhere to the type definitions established in the previous notebooks.

In [ ]:
import { RecursiveSet, Tuple, Value } from "recursive-set";
import { key } from "./05-DFA-2-RegExp";

## Type Definitions

We utilize the types established in our DFA implementation, specifically:
* **`DFAState`**: A state in our DFA (conceptually a set of original NFA states).
* **`DFA`**: The object structure representing the automaton.

For the minimization process, we introduce specific types to represent state equivalences:

* **`Pair`**: A tuple $(p, q)$ representing two states being compared.
* **`EquivClass`**: A set of `DFAState`s that are equivalent to each other (also known as $[q]$).
* **`Partition`**: A set of equivalence classes that partitions the entire state space $Q$.

In [ ]:
type State = string | number;
type Char = string;

type DFAState = RecursiveSet<State>; 

type TransRelDet = Map<string, DFAState>;

type DFA = {
  Q: RecursiveSet<DFAState>;
  Sigma: RecursiveSet<Char>;
  delta: TransRelDet;
  q0: DFAState;
  A: RecursiveSet<DFAState>;
};

// === Minimization Specific Types ===

type Pair = Tuple<[DFAState, DFAState]>;
type EquivClass = RecursiveSet<DFAState>;
type Partition = RecursiveSet<RecursiveSet<DFAState>>

### Helper Functions

**Arbitrary Element:**
The function `arb(M)` returns an *arbitrary* element from a set `M`.
Since `RecursiveSet` implements the iterator protocol and is internally sorted, this function deterministically returns the "smallest" element.

**Cartesian Product:**
The function `cartProd(A, B)` computes the Cartesian product $A \times B$. We leverage the built-in `cartesianProduct` method.

In [ ]:
function arb<T extends Value>(M: RecursiveSet<T>): T {
  if (M.isEmpty()) throw new Error("Error: arb called with empty set!");
  return M.raw[0];
}

function cartProd<S extends Value, T extends Value>(
  A: RecursiveSet<S>, 
  B: RecursiveSet<T>
): RecursiveSet<Tuple<[S, T]>> {
  return A.cartesianProduct(B);
}

### Separation Logic

The function `separate` takes four arguments:
- `Pairs`: A set of pairs known to be separable.
- `States`: The set of all states $Q$.
- `Sigma`: The alphabet $\Sigma$.
- `delta`: The transition map.

It returns a **new** set of separable pairs. Two states $q_1, q_2$ become separable if there exists a character $c$ such that their targets $p_1 = \delta(q_1, c)$ and $p_2 = \delta(q_2, c)$ are already known to be separable.

In [ ]:
function separate(
  Pairs: RecursiveSet<Pair>, 
  States: RecursiveSet<DFAState>, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<Pair> {
  
  const newPairsArr: Pair[] = [];
  
  const statesArr = States.raw;
  const sigmaArr = Sigma.raw;

  for (const q1 of statesArr) {
    for (const q2 of statesArr) {
      for (const c of sigmaArr) {
        const p1 = delta.get(key(q1, c));
        const p2 = delta.get(key(q2, c));
        
        if (p1 && p2) {
          const targetPair = new Tuple(p1, p2);
          
          if (Pairs.has(targetPair)) {
            newPairsArr.push(new Tuple(q1, q2));
          }
        }
      }
    }
  }
  return RecursiveSet.fromArray(newPairsArr);
}

### Computing All Separable Pairs

The function `allSeparable` computes the set of all pairs $(p, q)$ that are **distinguishable**.

It uses a **fixed-point iteration**:
1.  **Base Case:** Start with pairs where one state is accepting and the other is not: $(Q \setminus A) \times A \cup A \times (Q \setminus A)$.
2.  **Iteration:** Repeatedly apply `separate` to find pairs that lead to already separated pairs. Stop when no new pairs are added.

In [ ]:
function allSeparable(
  Q: RecursiveSet<DFAState>, 
  A: RecursiveSet<DFAState>, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<Pair> {
  
  const NonAccepting = Q.difference(A);

  const set1 = cartProd(NonAccepting, A);
  const set2 = cartProd(A, NonAccepting);
  
  let Separable = set1.union(set2);

  while (true) {
    const NewPairs = separate(Separable, Q, Sigma, delta);
    
    if (NewPairs.isSubset(Separable)) {
      return Separable;
    }
    
    Separable = Separable.union(NewPairs);
  }
}

### Reachability

The function `reachable` filters out states that cannot be reached from the start state $q_0$. Unreachable states are irrelevant for the language accepted by the DFA. It also uses a fixed-point iteration.

In [ ]:
function reachable(
  q0: DFAState, 
  Sigma: RecursiveSet<Char>, 
  delta: TransRelDet
): RecursiveSet<DFAState> {
  
  const visited = new RecursiveSet<DFAState>(q0);
  const queue: DFAState[] = [q0];
  
  let head = 0;
  while(head < queue.length) {
      const p = queue[head++];
      
      for (const c of Sigma) {
          const target = delta.get(key(p, c));
          if (target && !visited.has(target)) {
              visited.add(target);
              queue.push(target);
          }
      }
  }
  return visited;
}

### Partitioning the State Space

The core of the minimization process is identifying which states are equivalent.

The function `computeMinimizationPartition` performs the heavy lifting:
1.  **Filter Reachable States:** Unreachable states are removed as they do not affect the accepted language.
2.  **Compute Distinguishability:** It uses `allSeparable` to find all pairs of states that can be distinguished by some input string.
3.  **Construct Partition:** It groups states that are *not* distinguishable into sets.

**Output:**
It returns a `Partition` (a set of sets), where each inner set represents an **Equivalence Class** (e.g., `{ {1, 2}, {3} }`). This intermediate structure allows us to inspect which states are merged before we construct the final, normalized DFA.

In [ ]:
function computeMinimizationPartition(F: DFA): Partition {
  let { Q, Sigma, delta, q0, A } = F;
  
  Q = reachable(q0, Sigma, delta);
  const reachableA = A.intersection(Q);
  
  const Separable = allSeparable(Q, reachableA, Sigma, delta);
  
  const EquivClasses = new RecursiveSet<EquivClass>();
  const Processed = new RecursiveSet<DFAState>();

  for (const q of Q) {
    if (Processed.has(q)) continue;

    const cls = new RecursiveSet<DFAState>();
    for (const p of Q) {
      const pairToCheck = new Tuple(p, q);
      if (!Separable.has(pairToCheck)) {
        cls.add(p);
        Processed.add(p);
      }
    }
    EquivClasses.add(cls);
  }
  
  return EquivClasses;
}

### The Minimization Algorithm (Normalization)

The `minimize` function combines the partition logic with a **Normalization Step** to produce the final DFA.

1.  **Compute Partition:** It calls `computeMinimizationPartition` to obtain the equivalence classes.
2.  **Normalization (Union Strategy):**
    The standard mathematical definition of a minimized DFA uses the equivalence classes themselves as states (sets of sets). However, strict adherence to this would result in nested types like `RecursiveSet<RecursiveSet<RecursiveSet<State>>>`.

    To maintain strict type compatibility with our `DFA` definition (where a state is simply `RecursiveSet<State>`) while preserving the semantic history of merged states, we define the new states as the **union** of the original states in each equivalence class.

    For each equivalence class $C \in \text{Partition}$, the new state $q_{new}$ is defined as:
    $$q_{new} = \bigcup_{s \in C} s$$

    **Example:** If states $\{2\}$ and $\{6\}$ are equivalent (i.e., $C = \{\{2\}, \{6\}\}$), the new minimized state becomes $\{2, 6\}$. This preserves traceability in the generated graphs.

3.  **Reconstruction:**
    * **Start State:** The new start state is the union-state corresponding to the class containing $q_0$.
    * **Transitions:** For a new state $Q_{new}$ (derived from class $C$) and character $c$, the target is the new state corresponding to the class containing $\delta(rep, c)$, where $rep$ is any representative element of $C$.

In [ ]:
function minimize(F: DFA): DFA {
  const { q0, A, delta, Sigma } = F;
  
  const EquivClasses = computeMinimizationPartition(F);

  const classToNewState = new Map<string, DFAState>();
  const newStatesArr: DFAState[] = [];
  
  for (const cls of EquivClasses) {
      let mergedState = new RecursiveSet<State>();
      for (const oldState of cls) {
          mergedState = mergedState.union(oldState);
      }
      
      newStatesArr.push(mergedState);
      classToNewState.set(cls.toString(), mergedState);
  }

  const getNewStateForOld = (oldState: DFAState): DFAState => {
     for (const cls of EquivClasses) {
         if (cls.has(oldState)) {
             return classToNewState.get(cls.toString())!;
         }
     }
     throw new Error(`State ${oldState} lost during minimization`);
  };

  const newQ0 = getNewStateForOld(q0);

  const newAcceptArr: DFAState[] = [];
  for (const cls of EquivClasses) {
      const rep = arb(cls);
      if (A.has(rep)) {
          newAcceptArr.push(classToNewState.get(cls.toString())!);
      }
  }

  const newDelta: TransRelDet = new Map();
  for (const cls of EquivClasses) {
      const newState = classToNewState.get(cls.toString())!;
      const rep = arb(cls); 
      
      for (const c of Sigma) {
          const targetOld = delta.get(key(rep, c));
          if (targetOld) {
              const targetNew = getNewStateForOld(targetOld);
              newDelta.set(key(newState, c), targetNew);
          }
      }
  }
  
  return {
    Q: RecursiveSet.fromArray(newStatesArr),
    Sigma,
    delta: newDelta,
    q0: newQ0,
    A: RecursiveSet.fromArray(newAcceptArr)
  };
}